In [1]:
from unittest import result
from icecream import ic
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import os
import json
import subprocess
import itertools
from statistics import mean
from sklearn.metrics import mean_squared_error
import re
import tqdm
import glob
from sklearn.metrics import mean_squared_error


In [10]:
quanttb_output = glob.glob('mixed_infection_output/*.txt')
MSE_quantb= {'0595':[],'1090':[],'1585':[],'2080':[],'2575':[],'3070':[],'3565':[],'4060':[],'4555':[],'5050':[]}

for x in quanttb_output:
    df = pd.read_csv(x)
    # print(x)
    ratio = x.split('-')[2].split('.')[0]
    string1 = slice(0,2)
    string2 = slice(2,4)
    r1, r2 = int(ratio[string1])/100, int(ratio[string2])/100
    if r1 >= r2:
        pass
    else:
        r1, r2 = r2, r1        

    quanttb = df[' relabundance'].values.tolist()
    if len(quanttb) == 1:
        quanttb.append(0)
    quanttb = quanttb[:2]

    MSE_quantb[ratio].append(mean_squared_error([r1, r2], quanttb))

for k,v in MSE_quantb.items():
    MSE_quantb[k] = mean(v)

In [ ]:
tbp_output = glob.glob('/mnt/storage7/lwang/trial_tb_philippines/data/processed/seqtk_new/tb-profiler/results/*.results.json')

FOLDER_PATH = "/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/insilico_mix_new_0.99/mixture_results/profiles"
NAME_PATH = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/insilico_mix/mixture_results/'
        
from sklearn.metrics import mean_squared_error

designed_ratio = [[0.05,0.95],[0.10,0.90],[0.15,0.85],[0.20,0.80],[0.25,0.75],[0.30,0.70],[0.35,0.65],[0.40,0.60],[0.45,0.55],[0.50,0.50],[0.55,0.45],[0.60,0.40],[0.65,0.35],[0.70,0.30],[0.75,0.25],[0.80,0.20],[0.85,0.15],[0.90,0.10]]
MSE_tbp= {'0595':[],'1090':[],'1585':[],'2080':[],'2575':[],'3070':[],'3565':[],'4060':[],'4555':[],'5050':[]}

tb = []
mse_tb = []
labels = ['0595','1090','1585','2080','2575','3070','3565','4060','4555','5050']

for x in tbp_output:
    ratio = x.split('-')[3].split('.')[0]
    string1 = slice(0,2)
    string2 = slice(2,4)
    r1, r2 = int(ratio[string1])/100, int(ratio[string2])/100
    if r1 >= r2:
        pass
    else:
        r1, r2 = r2, r1       
    
    
    FILE_PATH = x
    json_results = json.load(open(FILE_PATH))
    tb_pred =[]  # put lineage 1 and lineage 4 in here - the labels uses ratio of lineage4-lineage1 but the output of gmm model always outputs the lineage with higher proportion first
    sublin = json_results['sublin'].split(';')
    tbp_freq = []

    for s in sublin:
        for x in json_results['lineage']:
            if x['lin'] == s:
                tbp_freq.append(x['frac'])
    if len(tbp_freq) == 1:
        tbp_freq.append(0)
    if tbp_freq[0] > tbp_freq[1]:
        pass
    else:
        tbp_freq[0], tbp_freq[1] = tbp_freq[1], tbp_freq[0]            
    
    MSE_tbp[ratio].append(mean_squared_error([r1, r2], tbp_freq))
    
for k,v in MSE_tbp.items():
    print(v)
    MSE_tbp[k] = mean(v)



In [55]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=labels, y=mse_tb, mode='lines', name = 'TB-profiler', line=dict(color="red")))
fig.add_trace(go.Scatter(x=list(MSE_quantb.keys()),y=list(MSE_quantb.values()), mode='lines',name = 'Quant-tb', line=dict(color="blue")))
fig.add_trace(go.Scatter(x=list(MSE_tbp.keys()),y=list(MSE_tbp.values()), mode='lines',name = 'tb-profiler', line=dict(color="red")))

fig.update_layout(
    # xaxis_range = [0.00010475137486554342, 0.02],
    title="In Silico model prediction MSE",
    xaxis_title="Strain Mix Ratio",
    yaxis_title="MSE",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

In [13]:
quanttb

[0.431, 0.569]

In [ ]:
from sklearn.metrics import mean_squared_error

mse_tb = {}
mse_gmm = {}


for x in json_names:
    sample = x.split('-')[2].split('.')[0]
    # print(sample)
    mse_tb[sample] = []
    mse_gmm[sample] = []

for x in json_names:
    sample = x.split('-')[2].split('.')[0]
    # print(sample)

    string1 = slice(0,2)
    string2 = slice(2,4)
    r1, r2 = int(sample[string1])/100, int(sample[string2])/100
    
    FILE_PATH_MIX = os.path.join(FOLDER_PATH,x)
    json_results = json.load(open(FILE_PATH_MIX))
    # tb_pred =[]  # put lineage 1 and lineage 4 in here - the labels uses ratio of lineage4-lineage1 but the output of gmm model always outputs the lineage with higher proportion first
    # gmm_pred=[]
    
    tb_temp = []
    gmm_temp = []
    for x in json_results["lineage"]:
        tb_temp.append(x['tb_pred'])
        gmm_temp.append(x['gmm_pred'])
    if tb_temp[0] > tb_temp[1]: #making sure the smaller number is place in front, aligning with the sample label order
        tb_temp[0], tb_temp[1] = tb_temp[1], tb_temp[0]
    # print(tb_temp)
    # print([r1, r2])
    mse = mean_squared_error([r1, r2], tb_temp)
    mse_tb[sample].append(mse)
    
    if gmm_temp[0] > gmm_temp[1]: #making sure the smaller number is place in front, aligning with the sample label order
        gmm_temp[0], gmm_temp[1] = gmm_temp[1], gmm_temp[0]
    mse = mean_squared_error([r1, r2], gmm_temp)
    mse_gmm[sample].append(mse)
    
    # tb_pred.append(x['tb_pred'])        
    # for x in json_results["lineage"]:
    #     gmm_pred.append(x['gmm_pred'])
    # tb.append(tb_pred)
    # gmm.append(gmm_pred) # append the list of lin pred to larger list

for k, v in mse_tb.items():
    mse_tb[k] = mean(v)

for k, v in mse_gmm.items():
    mse_gmm[k] = mean(v)

# Mixed Clinical samples comparison

In [2]:
quanttb_clinical_mix_output = glob.glob('/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/QuantTB/mixed_clinical_samples/results/output/*.txt')

mix_list = []

for x in quanttb_clinical_mix_output:
    df = pd.read_csv(x)
    # print(x)
    # ratio = x.split('-')[2].split('.')[0]
    # string1 = slice(0,2)
    # string2 = slice(2,4)
    # r1, r2 = int(ratio[string1])/100, int(ratio[string2])/100
    # if r1 >= r2:
    #     pass
    # else:
    #     r1, r2 = r2, r1        

    list_ = df[' relabundance'].values.tolist()
    list_.sort(reverse=True)

    mix_list.append(list_)

#     quanttb = df[' relabundance'].values.tolist()
#     if len(quanttb) == 1:
#         quanttb.append(0)
#     quanttb = quanttb[:2]

#     MSE_quantb[ratio].append(mean_squared_error([r1, r2], quanttb))

# for k,v in MSE_quantb.items():
#     MSE_quantb[k] = mean(v)

In [4]:
clinical_mix_sample_list = []
for x in quanttb_clinical_mix_output:
    clinical_mix_sample_list.append(x.split('/')[11].split('.')[0])


In [5]:
gmm_output_path = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/multi_infection_clinical/mixture_results/profiles'
# gmm_output_path = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/2mix_clinical_new_multi/mixture_results/profiles'
tb_pred_list = []

for x in clinical_mix_sample_list:
    FILE_PATH_MIX = os.path.join(gmm_output_path,x)
    FILE_PATH_MIX = FILE_PATH_MIX+'.gatk.mix.json'
    json_results = json.load(open(FILE_PATH_MIX))

    tb_pred = []
    for x in json_results['lineage']:
        tb_pred.append(x['tb_pred'])
    tb_pred.sort(reverse=True)
    tb_pred_list.append(tb_pred)

In [63]:
# gmm_output_path = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/multi_infection_clinical/mixture_results/profiles'
# # gmm_output_path = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/2mix_clinical_new_multi/mixture_results/profiles'
# tb_pred_list = []

# for x in clinical_mix_sample_list:
#     FILE_PATH_MIX = os.path.join(gmm_output_path,x)
#     FILE_PATH_MIX = FILE_PATH_MIX+'.gatk.mix.json'
#     json_results = json.load(open(FILE_PATH_MIX))

#     tb_pred = []
#     for x in json_results['lineage']:
#         tb_pred.append(x['tb_pred'])
#     tb_pred.sort(reverse=True)
#     tb_pred_list.append(tb_pred)

In [15]:
tb_p_mix_no = []
for x in tb_pred_list:
    tb_p_mix_no.append(len(x))
print(np.unique(tb_p_mix_no))

[2]


In [16]:
qtb_mix_no = []
for x in mix_list:
    qtb_mix_no.append(len(x))
print(np.unique(tb_p_mix_no))

[2]


In [ ]:
tb_p_mix_no

In [19]:
list_ = []
for x, y in zip(tb_p_mix_no, qtb_mix_no):
    if y >= x:
        list_.append(True)
    else:
        list_.append(False)


In [22]:
list_.count(True)/len(list_)

0.9200743494423792

# Unused

In [60]:
list_ = []
with open('/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/2mix_clinical_new_multi/mixture_results/profiles/clinical_mixed_sample.txt', 'r') as file:
    lines = file.readlines()
    for x in lines:
        list_.append(x.strip())

In [61]:
gmm_output_path = '/mnt/storage7/lwang/trial_tb_philippines/pipelines/Genomic_data_analysis/Executable_eval/results/2mix_clinical_new_multi/mixture_results/profiles'
tb_pred_list = []

for x in list_:
    FILE_PATH_MIX = os.path.join(gmm_output_path,x)
    FILE_PATH_MIX = FILE_PATH_MIX+'.gatk.mix.json'
    json_results = json.load(open(FILE_PATH_MIX))

    tb_pred = []
    for x in json_results['lineage']:
        tb_pred.append(x['tb_pred'])
    tb_pred.sort(reverse=True)
    tb_pred_list.append(tb_pred)

In [62]:
tb_p_mix_no = []
for x in tb_pred_list:
    tb_p_mix_no.append(len(x))
print(np.unique(tb_p_mix_no))

[2]
